
# **Laboratorio de aprendizaje por refuerzo**

## Introducción

El aprendizaje por refuerzo es un enfoque general para aprender una política de control en sistemas dinámicos estocásticos a través de la interacción. En este paradigma, un agente aprende a optimizar su comportamiento acumulando recompensas con el tiempo. A diferencia de la optimización local, el aprendizaje por refuerzo se enfoca en descubrir el control óptimo incluso cuando las recompensas están diferidas en el tiempo.

En este laboratorio, exploraremos dos algoritmos clave:
1. **SARSA** (*State-Action-Reward-State-Action*): Un algoritmo *on-policy* que actualiza los valores de las acciones en función de la política que sigue el agente.
2. **Q-Learning**: Un algoritmo *off-policy* que estima el valor de la política óptima independientemente de la política actual del agente.

Además, abordaremos el dilema de **exploración-explotación**, donde el agente debe balancear entre:
- **Explorar**: Probar nuevas acciones para descubrir más sobre el entorno.
- **Explotar**: Utilizar lo aprendido para maximizar las recompensas conocidas.

Para simplificar el aprendizaje de los algoritmos, utilizaremos el entorno **FrozenLake-v0** de OpenAI Gym, un problema sencillo que permite ilustrar los conceptos básicos del aprendizaje por refuerzo.


## Marco Teórico

### **Elementos del Aprendizaje por Refuerzo**
1. **Agente**: Entidad que toma decisiones.
2. **Entorno**: Sistema en el que opera el agente.
3. **Estados ($ S $)**: Configuraciones posibles del entorno.
4. **Acciones ($ A $)**: Conjunto de decisiones posibles para el agente.
5. **Recompensas ($ R $)**: Retroalimentación numérica que guía al agente.
6. **Política ($ \pi $)**: Estrategia que sigue el agente para tomar decisiones.

### **Elementos del Aprendizaje por Refuerzo en el Contexto de FrozenLake**

#### 1. **Agente**:
- **Definición**: El agente es quien toma decisiones en el entorno para maximizar la recompensa acumulada.
- **En FrozenLake**: 
  - El agente es el jugador que debe moverse desde la posición inicial `S` (Start) hasta la meta `G` (Goal) evitando caer en los agujeros `H`.

---

#### 2. **Entorno**:
- **Definición**: El sistema en el que opera el agente. Proporciona estados y recompensas en respuesta a las acciones tomadas por el agente.
- **En FrozenLake**:
  - El entorno es el mapa representado como una cuadrícula:
    - `S`: Inicio.
    - `F`: Hielo congelado (camino seguro).
    - `H`: Agujeros (estado terminal negativo).
    - `G`: Meta (estado terminal positivo).

---

#### 3. **Estados ($ S $)**:
- **Definición**: Representan las configuraciones posibles del entorno en un momento dado.
- **En FrozenLake**:
  - Los estados corresponden a las posiciones del agente en la cuadrícula.
  - Por ejemplo, en un mapa de 4x4, hay $ 16 $ estados posibles numerados de $ 0 $ a $ 15 $.
  - El estado se calcula como:
    \[
    \text{Estado} = \text{fila} \times \text{número de columnas} + \text{columna}
    \]

---

#### 4. **Acciones ($ A $)**:
- **Definición**: Conjunto de decisiones que el agente puede tomar desde un estado dado.
- **En FrozenLake**:
  - El agente tiene 4 acciones disponibles:
    - `0`: Izquierda.
    - `1`: Abajo.
    - `2`: Derecha.
    - `3`: Arriba.
  - En modo resbaladizo (`is_slippery=True`), la acción tomada puede no coincidir con el movimiento efectivo debido a la estocasticidad.

---

#### 5. **Recompensas ($ R $)**:
- **Definición**: Retroalimentación numérica que guía al agente hacia su objetivo.
- **En FrozenLake**:
  - Las recompensas están definidas como:
    - **Meta (`G`)**: $ +1 $.
    - **Hielo (`F`) o Inicio (`S`)**: $ 0 $.
    - **Agujero (`H`)**: $ 0 $.

---

#### 6. **Política ($ \pi $)**:
- **Definición**: Estrategia que sigue el agente para tomar decisiones en cada estado.
- **En FrozenLake**:
  - La política puede ser:
    - **Inicial**: Selección aleatoria de acciones (exploración).
    - **Aprendida**: Después de entrenar al agente, la política selecciona la acción con el mayor valor $ Q(s, a) $ en cada estado.


### **Algoritmos de Aprendizaje**

1. **SARSA**:
   - SARSA es un algoritmo *on-policy*, lo que significa que actualiza los valores $ Q(s, a) $ basándose en la política actual.
   - Ecuación de actualización:
     $$
     Q_{t+1}(s_t, a_t) \leftarrow Q_t(s_t, a_t) + \alpha \left[ r_t + \gamma Q_t(s_{t+1}, a_{t+1}) - Q_t(s_t, a_t) \right]
     $$
     Donde:
     - $ \alpha $: Tasa de aprendizaje.
     - $ \gamma $: Factor de descuento.
     - $ r_t $: Recompensa en el paso $ t $.

2. **Q-Learning**:
   - Q-Learning es un algoritmo *off-policy* que busca la política óptima sin seguir necesariamente la política actual.
   - Ecuación de actualización:
     $$
     Q_{t+1}(s_t, a_t) \leftarrow Q_t(s_t, a_t) + \alpha \left[ r_t + \gamma \max_b Q_t(s_{t+1}, b) - Q_t(s_t, a_t) \right]
     $$


### **Estrategias de Exploración**
1. **ε-greedy**:
   - Con probabilidad $ 1-\epsilon $, el agente elige la mejor acción conocida.
   - Con probabilidad $ \epsilon $, selecciona una acción aleatoria para explorar.
2. **Softmax**:
   - Asigna probabilidades a cada acción según sus valores $ Q(s, a) $:
     $$
     P(a_i|s) = \frac{e^{\frac{Q(s, a_i)}{\tau}}}{\sum_j e^{\frac{Q(s, a_j)}{\tau}}}
     $$
     Donde $ \tau $ controla el nivel de exploración.

### **Evaluación y Rendimiento**
1. Para evaluar el aprendizaje, es esencial contar los éxitos en ventanas de episodios (por ejemplo, 100).
2. Las políticas aprendidas deben evaluarse repetidamente para obtener un promedio confiable, especialmente en entornos estocásticos.


In [ ]:
#Reglas del juego
import random

class FrozenLake:
    def __init__(self, map_desc=None, is_slippery=True):
        self.map_desc = map_desc or [
            "SFFF",
            "FHFH",
            "FFFH",
            "HFFG"
        ]
        self.is_slippery = is_slippery
        self.n_rows = len(self.map_desc)
        self.n_cols = len(self.map_desc[0])
        self.start_state = self._find_start()
        self.state = self.start_state
        self.terminated = False

        # Diccionario de movimientos (acciones) corregido
        self.actions = {
            0: (0, -1),  # Izquierda
            1: (1, 0),   # Abajo
            2: (0, 1),   # Derecha
            3: (-1, 0)   # Arriba
        }

    def _find_start(self):
        """Encuentra la posición inicial (S) en el mapa."""
        for row in range(self.n_rows):
            for col in range(self.n_cols):
                if self.map_desc[row][col] == "S":
                    return (row, col)
        raise ValueError("Mapa no contiene estado inicial 'S'.")

    def reset(self):
        """Reinicia el juego al estado inicial."""
        self.state = self.start_state
        self.terminated = False
        return self.state

    def _inside_bounds(self, row, col):
        """Verifica si la posición está dentro del mapa."""
        return 0 <= row < self.n_rows and 0 <= col < self.n_cols

    def step(self, action):
        """Aplica una acción y devuelve el nuevo estado, recompensa, y si terminó."""
        if self.terminated:
            raise ValueError("El episodio ya terminó. Reinicia el juego con reset().")

        # Probabilidad de resbalar
        if self.is_slippery:
            action = random.choice(list(self.actions.keys()))

        # Determinar la nueva posición
        move = self.actions[action]
        new_row = self.state[0] + move[0]
        new_col = self.state[1] + move[1]

        # Si la nueva posición está fuera de los límites, el agente no se mueve
        if not self._inside_bounds(new_row, new_col):
            new_row, new_col = self.state

        # Actualizar estado
        new_state = (new_row, new_col)
        tile = self.map_desc[new_row][new_col]

        # Determinar recompensa y si terminó el episodio
        reward = 0
        if tile == "H":  # Agujero
            self.terminated = True
        elif tile == "G":  # Meta
            reward = 1
            self.terminated = True

        self.state = new_state
        return new_state, reward, self.terminated

    def render(self):
        """Muestra el mapa actual con la posición del agente."""
        for row in range(self.n_rows):
            row_str = ""
            for col in range(self.n_cols):
                if (row, col) == self.state:
                    row_str += "A"  # Agente
                else:
                    row_str += self.map_desc[row][col]
            print(row_str)
        print()


## **Ejercicios**

### **Ejercicio 1: Familiarización con OpenAI Gym**
Este ejercicio tiene como propósito entender cómo interactuar con el entorno FrozenLake-v1 y explorar sus características principales. Esto incluye:
- **Objetivo**:
  - Familiarizarse con los espacios de observación y acción.
  - Observar cómo las acciones afectan el estado y la recompensa.
  - Experimentar con los parámetros clave, como:
    - Mapas personalizados.
    - Naturaleza resbaladiza del entorno.

In [12]:
import tkinter as tk
from PIL import Image, ImageTk
import os
import random

class FrozenLakeVisualizer:
    def __init__(self, root, lake):
        self.root = root
        self.lake = lake
        self.tile_size = 100  # Tamaño de cada celda en píxeles
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.state = lake.reset()
        self.done = False

        # Cargar las imágenes
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.action_buttons = {
            "Izquierda": 0,
            "Abajo": 1,
            "Derecha": 2,
            "Arriba": 3
        }
        self.create_buttons()

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona un botón para mover al agente.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img' y las redimensiona."""
        self.images = {}
        img_folder = "img"

        try:
            # Cargar imágenes de celdas
            self.images["S"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["F"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["H"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["G"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
            )

            # Cargar imágenes del agente
            self.images["elf_down"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["elf_left"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_left.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["elf_right"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_right.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["elf_up"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_up.png")).resize((self.tile_size, self.tile_size))
            )

        except FileNotFoundError as e:
            print(f"Error al cargar imágenes: {e}")
            print("Asegúrate de que la carpeta 'img' y las imágenes necesarias existan.")
            exit()

    def create_buttons(self):
        """Crea los botones de acción."""
        self.button_frame = tk.Frame(self.root)
        self.button_frame.pack()
        for action, code in self.action_buttons.items():
            button = tk.Button(self.button_frame, text=action, command=lambda c=code: self.take_step(c))
            button.pack(side=tk.LEFT, padx=10)

        # Botón para reiniciar
        self.reset_button = tk.Button(self.root, text="Reiniciar", command=self.reset_game)
        self.reset_button.pack(pady=10)

    def draw_map(self):
        """Dibuja el mapa del entorno con imágenes redimensionadas."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual con la imagen correspondiente."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size

        # Seleccionar la imagen del agente según la acción
        action_to_image = {
            0: "elf_left",
            1: "elf_down",
            2: "elf_right",
            3: "elf_up"
        }
        agent_image = action_to_image.get(getattr(self, "last_action", 1), "elf_down")
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images[agent_image])

    def take_step(self, action):
        """Realiza un paso en el juego."""
        if self.done:
            self.message_label.config(text="El episodio ya terminó. Presiona 'Reiniciar' para empezar de nuevo.")
            return

        # Guardar la última acción para la imagen del agente
        self.last_action = action

        # Realizar el paso
        next_state, reward, terminated = self.lake.step(action)
        self.state = next_state
        self.done = terminated
        self.draw_map()

        # Actualizar mensajes
        if self.done:
            if reward > 0:
                self.message_label.config(text="¡Ganaste! Llegaste a la meta.")
            else:
                self.message_label.config(text="¡Perdiste! Caíste en un agujero.")
        else:
            self.message_label.config(text=f"Estado: {self.state}, Acción: {action}, Recompensa: {reward}")

    def reset_game(self):
        """Reinicia el juego."""
        self.state = self.lake.reset()
        self.done = False
        self.last_action = None  # No hay acción previa al reiniciar
        self.draw_map()
        self.message_label.config(text="Presiona un botón para mover al agente.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=False)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake con Imágenes Redimensionadas")
app = FrozenLakeVisualizer(root, lake)
root.mainloop()


In [9]:
import tkinter as tk
from PIL import Image, ImageTk
import os
import random

class FrozenLakeVisualizer:
    def __init__(self, root, lake):
        self.root = root
        self.lake = lake
        self.tile_size = 100  # Tamaño de cada celda en píxeles
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.state = lake.reset()
        self.done = False

        # Cargar las imágenes
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.action_buttons = {
            "Izquierda": 0,
            "Abajo": 1,
            "Derecha": 2,
            "Arriba": 3
        }
        self.create_buttons()

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona un botón para mover al agente.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img' y las redimensiona."""
        self.images = {}
        img_folder = "img"

        try:
            # Cargar imágenes de celdas
            self.images["S"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["F"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["H"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["G"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
            )

            # Cargar imágenes del agente
            self.images["elf_down"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["elf_left"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_left.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["elf_right"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_right.png")).resize((self.tile_size, self.tile_size))
            )
            self.images["elf_up"] = ImageTk.PhotoImage(
                Image.open(os.path.join(img_folder, "elf_up.png")).resize((self.tile_size, self.tile_size))
            )

        except FileNotFoundError as e:
            print(f"Error al cargar imágenes: {e}")
            print("Asegúrate de que la carpeta 'img' y las imágenes necesarias existan.")
            exit()

    def create_buttons(self):
        """Crea los botones de acción."""
        self.button_frame = tk.Frame(self.root)
        self.button_frame.pack()
        for action, code in self.action_buttons.items():
            button = tk.Button(self.button_frame, text=action, command=lambda c=code: self.take_step(c))
            button.pack(side=tk.LEFT, padx=10)

        # Botón para reiniciar
        self.reset_button = tk.Button(self.root, text="Reiniciar", command=self.reset_game)
        self.reset_button.pack(pady=10)

    def draw_map(self):
        """Dibuja el mapa del entorno con imágenes redimensionadas."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual con la imagen correspondiente."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size

        # Seleccionar la imagen del agente según la acción
        action_to_image = {
            0: "elf_left",
            1: "elf_down",
            2: "elf_right",
            3: "elf_up"
        }
        agent_image = action_to_image.get(getattr(self, "last_action", 1), "elf_down")
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images[agent_image])

    def take_step(self, action):
        """Realiza un paso en el juego."""
        if self.done:
            self.message_label.config(text="El episodio ya terminó. Presiona 'Reiniciar' para empezar de nuevo.")
            return

        # Guardar la última acción para la imagen del agente
        self.last_action = action

        # Realizar el paso
        next_state, reward, terminated = self.lake.step(action)
        self.state = next_state
        self.done = terminated
        self.draw_map()

        # Actualizar mensajes
        if self.done:
            if reward > 0:
                self.message_label.config(text="¡Ganaste! Llegaste a la meta.")
            else:
                self.message_label.config(text="¡Perdiste! Caíste en un agujero.")
        else:
            self.message_label.config(text=f"Estado: {self.state}, Acción: {action}, Recompensa: {reward}")

    def reset_game(self):
        """Reinicia el juego."""
        self.state = self.lake.reset()
        self.done = False
        self.last_action = None  # No hay acción previa al reiniciar
        self.draw_map()
        self.message_label.config(text="Presiona un botón para mover al agente.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake con Imágenes Redimensionadas")
app = FrozenLakeVisualizer(root, lake)
root.mainloop()



### **Ejercicio 2: SARSA**
- **Objetivo**:
  - Implementar el algoritmo SARSA con exploración **ε-greedy**.
  - Actualizar una tabla $ Q $ de valores para cada estado y acción.
  - Medir el rendimiento contando los episodios exitosos en ventanas de 100 episodios.
  - Considerar el problema resuelto si la tasa de éxito supera el 76%.

In [4]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import random


class SARSAVisualizer:
    def __init__(self, root, lake, episodes=500):
        self.root = root
        self.lake = lake
        self.episodes = episodes
        self.alpha = 0.1  # Tasa de aprendizaje
        self.gamma = 0.99  # Factor de descuento
        self.epsilon = 0.5  # Probabilidad inicial de exploración
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Tabla Q

        # Configuración del episodio actual
        self.current_episode = 0
        self.state = self.lake.reset()
        self.action = None
        self.done = False
        self.success_count = 0
        self.success_history = []  # Historial de éxitos (en ventanas de 100 episodios)

        # Configuración de la interfaz gráfica
        self.tile_size = 100
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.step_button = tk.Button(root, text="Ejecutar SARSA", command=self.run_sarsa)
        self.step_button.pack()

        self.reset_button = tk.Button(root, text="Reiniciar", command=self.reset)
        self.reset_button.pack(pady=10)

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona 'Ejecutar SARSA' para comenzar.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img'."""
        self.images = {}
        img_folder = "img"
        self.images["S"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["F"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["H"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["G"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["elf_down"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
        )

    def draw_map(self):
        """Dibuja el mapa del entorno."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images["elf_down"])

    def epsilon_greedy(self, state):
        """Selecciona una acción usando ε-greedy."""
        if np.random.uniform(0, 1) < self.epsilon:
            return random.choice([0, 1, 2, 3])  # Exploración
        else:
            return np.argmax(self.q_table[state])  # Explotación

    def run_sarsa(self):
        """Ejecuta el algoritmo SARSA."""
        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            action = self.epsilon_greedy(state_index)
            done = False
            step = 0

            while not done:
                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]
                next_action = self.epsilon_greedy(next_state_index)

                # Actualizar la tabla Q usando la regla de SARSA
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * self.q_table[next_state_index, next_action]
                    - self.q_table[state_index, action]
                )

                # Actualizar estado y acción
                state_index, action = next_state_index, next_action
                self.state = next_state
                done = terminated
                step += 1

                # Redibujar el mapa
                self.draw_map()
                self.root.update()

                # Verificar si se alcanzó el objetivo
                if terminated and reward > 0:
                    self.success_count += 1

            # Reducir ε después de cada episodio
            self.epsilon = max(0.01, self.epsilon * 0.995)

            # Actualizar el historial de éxitos cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = self.success_count / 100
                self.success_history.append(success_rate)
                self.success_count = 0

        # Mostrar resultados finales
        self.message_label.config(text=f"SARSA completado. Tasa de éxito: {self.success_history[-1]:.2f}")

    def reset(self):
        """Reinicia el entorno."""
        self.state = self.lake.reset()
        self.q_table = np.zeros((self.lake.n_rows * self.lake.n_cols, 4))
        self.current_episode = 0
        self.done = False
        self.success_count = 0
        self.success_history = []
        self.draw_map()
        self.message_label.config(text="Presiona 'Ejecutar SARSA' para comenzar.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake - Ejercicio 2: SARSA")
app = SARSAVisualizer(root, lake, episodes=500)
root.mainloop()


### **Ejercicio 3: Q-Learning**
- **Objetivo**:
  - Sustituir la regla de actualización de SARSA por la de Q-Learning.
  - Comparar el rendimiento entre SARSA y Q-Learning para analizar sus diferencias.


In [3]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import random


class QLearningVisualizer:
    def __init__(self, root, lake, episodes=500):
        self.root = root
        self.lake = lake
        self.episodes = episodes
        self.alpha = 0.5  # Tasa de aprendizaje
        self.gamma = 0.99  # Factor de descuento
        self.epsilon = 0.5  # Probabilidad inicial de exploración
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Tabla Q

        # Configuración del episodio actual
        self.current_episode = 0
        self.state = self.lake.reset()
        self.done = False
        self.success_count = 0
        self.success_history = []  # Historial de éxitos (en ventanas de 100 episodios)

        # Configuración de la interfaz gráfica
        self.tile_size = 100
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.step_button = tk.Button(root, text="Ejecutar Q-Learning", command=self.run_q_learning)
        self.step_button.pack()

        self.reset_button = tk.Button(root, text="Reiniciar", command=self.reset)
        self.reset_button.pack(pady=10)

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona 'Ejecutar Q-Learning' para comenzar.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img'."""
        self.images = {}
        img_folder = "img"
        self.images["S"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["F"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["H"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["G"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["elf_down"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
        )

    def draw_map(self):
        """Dibuja el mapa del entorno."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images["elf_down"])

    def epsilon_greedy(self, state):
        """Selecciona una acción usando ε-greedy."""
        if np.random.uniform(0, 1) < self.epsilon:
            return random.choice([0, 1, 2, 3])  # Exploración
        else:
            return np.argmax(self.q_table[state])  # Explotación

    def run_q_learning(self):
        """Ejecuta el algoritmo Q-Learning."""
        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False
            step = 0

            while not done:
                # Seleccionar acción usando ε-greedy
                action = self.epsilon_greedy(state_index)

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Actualizar la tabla Q usando la regla de Q-Learning
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                # Actualizar estado
                state_index = next_state_index
                self.state = next_state
                done = terminated
                step += 1

                # Redibujar el mapa
                self.draw_map()
                self.root.update()

                # Verificar si se alcanzó el objetivo
                if terminated and reward > 0:
                    self.success_count += 1

            # Reducir ε después de cada episodio
            self.epsilon = max(0.01, self.epsilon * 0.995)

            # Actualizar el historial de éxitos cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = self.success_count / 100
                self.success_history.append(success_rate)
                self.success_count = 0

        # Mostrar resultados finales
        self.message_label.config(text=f"Q-Learning completado. Tasa de éxito: {self.success_history[-1]:.2f}")

    def reset(self):
        """Reinicia el entorno."""
        self.state = self.lake.reset()
        self.q_table = np.zeros((self.lake.n_rows * self.lake.n_cols, 4))
        self.current_episode = 0
        self.done = False
        self.success_count = 0
        self.success_history = []
        self.draw_map()
        self.message_label.config(text="Presiona 'Ejecutar Q-Learning' para comenzar.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake - Ejercicio 3: Q-Learning")
app = QLearningVisualizer(root, lake, episodes=500)
root.mainloop()


### **Ejercicio 4: Exploración Softmax**
- **Objetivo**:
  - Implementar una estrategia de exploración Softmax.
  - Asignar probabilidades de selección de acciones proporcionalmente a sus valores $ Q(s, a) $.

In [7]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import random


class SoftmaxQLearningVisualizer:
    def __init__(self, root, lake, episodes=500, tau=1.0):
        self.root = root
        self.lake = lake
        self.episodes = episodes
        self.tau = tau  # Temperatura inicial
        self.alpha = 0.5  # Tasa de aprendizaje
        self.gamma = 0.99  # Factor de descuento
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Tabla Q

        # Configuración del episodio actual
        self.current_episode = 0
        self.state = self.lake.reset()
        self.done = False
        self.success_count = 0
        self.success_history = []  # Historial de éxitos (en ventanas de 100 episodios)

        # Configuración de la interfaz gráfica
        self.tile_size = 100
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.step_button = tk.Button(root, text="Ejecutar Softmax Q-Learning", command=self.run_softmax_q_learning)
        self.step_button.pack()

        self.reset_button = tk.Button(root, text="Reiniciar", command=self.reset)
        self.reset_button.pack(pady=10)

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona 'Ejecutar Softmax Q-Learning' para comenzar.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img'."""
        self.images = {}
        img_folder = "img"
        self.images["S"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["F"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["H"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["G"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["elf_down"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
        )

    def draw_map(self):
        """Dibuja el mapa del entorno."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images["elf_down"])

    def softmax(self, state):
        """Calcula las probabilidades de las acciones usando Softmax."""
        q_values = self.q_table[state]
        exp_values = np.exp(q_values / self.tau)
        probabilities = exp_values / np.sum(exp_values)
        return probabilities

    def select_action_softmax(self, state):
        """Selecciona una acción basada en las probabilidades Softmax."""
        probabilities = self.softmax(state)
        return np.random.choice(len(probabilities), p=probabilities)

    def run_softmax_q_learning(self):
        """Ejecuta el algoritmo Q-Learning con exploración Softmax."""
        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False
            step = 0

            while not done:
                # Seleccionar acción usando Softmax
                action = self.select_action_softmax(state_index)

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Actualizar la tabla Q usando la regla de Q-Learning
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                # Actualizar estado
                state_index = next_state_index
                self.state = next_state
                done = terminated
                step += 1

                # Redibujar el mapa
                self.draw_map()
                self.root.update()

                # Verificar si se alcanzó el objetivo
                if terminated and reward > 0:
                    self.success_count += 1

            # Reducir temperatura después de cada episodio
            self.tau = max(0.1, self.tau * 0.995)

            # Actualizar el historial de éxitos cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = self.success_count / 100
                self.success_history.append(success_rate)
                self.success_count = 0

        # Mostrar resultados finales
        self.message_label.config(text=f"Softmax Q-Learning completado. Tasa de éxito: {self.success_history[-1]:.2f}")

    def reset(self):
        """Reinicia el entorno."""
        self.state = self.lake.reset()
        self.q_table = np.zeros((self.lake.n_rows * self.lake.n_cols, 4))
        self.current_episode = 0
        self.done = False
        self.success_count = 0
        self.success_history = []
        self.tau = 1.0  # Restaurar temperatura inicial
        self.draw_map()
        self.message_label.config(text="Presiona 'Ejecutar Softmax Q-Learning' para comenzar.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake - Ejercicio 4: Softmax Q-Learning")
app = SoftmaxQLearningVisualizer(root, lake, episodes=500, tau=1.0)
root.mainloop()


### **Ejercicio 5: Decaimiento de ε**
- **Objetivo**:
  - Diseñar un esquema para reducir $ \epsilon $ con el tiempo.
  - Evaluar cómo mejora la política aprendida en comparación con la política de exploración.


In [10]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import os
import random


class DecayEpsilonQLearningVisualizer:
    def __init__(self, root, lake, episodes=500, epsilon_0=0.5, epsilon_min=0.01, decay=0.995):
        self.root = root
        self.lake = lake
        self.episodes = episodes
        self.epsilon = epsilon_0  # Valor inicial de epsilon
        self.epsilon_min = epsilon_min  # Valor mínimo de epsilon
        self.decay = decay  # Factor de decaimiento
        self.alpha = 0.5  # Tasa de aprendizaje
        self.gamma = 0.99  # Factor de descuento
        self.q_table = np.zeros((lake.n_rows * lake.n_cols, 4))  # Tabla Q

        # Configuración del episodio actual
        self.current_episode = 0
        self.state = self.lake.reset()
        self.done = False
        self.success_count = 0
        self.success_history = []  # Historial de éxitos (en ventanas de 100 episodios)

        # Configuración de la interfaz gráfica
        self.tile_size = 100
        self.grid_size = (lake.n_rows, lake.n_cols)
        self.load_images()

        # Crear el canvas para el mapa
        self.canvas = tk.Canvas(
            root, width=self.grid_size[1] * self.tile_size, height=self.grid_size[0] * self.tile_size
        )
        self.canvas.pack()
        self.draw_map()

        # Botones
        self.step_button = tk.Button(root, text="Ejecutar Q-Learning con Decaimiento de ε", command=self.run_q_learning)
        self.step_button.pack()

        self.reset_button = tk.Button(root, text="Reiniciar", command=self.reset)
        self.reset_button.pack(pady=10)

        # Mensajes
        self.message_label = tk.Label(root, text="Presiona 'Ejecutar Q-Learning con Decaimiento de ε' para comenzar.")
        self.message_label.pack()

    def load_images(self):
        """Carga las imágenes necesarias desde la carpeta 'img'."""
        self.images = {}
        img_folder = "img"
        self.images["S"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["F"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "ice.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["H"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "hole.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["G"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "goal.png")).resize((self.tile_size, self.tile_size))
        )
        self.images["elf_down"] = ImageTk.PhotoImage(
            Image.open(os.path.join(img_folder, "elf_down.png")).resize((self.tile_size, self.tile_size))
        )

    def draw_map(self):
        """Dibuja el mapa del entorno."""
        self.canvas.delete("all")  # Limpiar el canvas
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                x, y = j * self.tile_size, i * self.tile_size
                tile = self.lake.map_desc[i][j]
                self.canvas.create_image(x, y, anchor=tk.NW, image=self.images[tile])

        # Dibujar al agente
        self.draw_agent()

    def draw_agent(self):
        """Dibuja al agente en la posición actual."""
        x, y = self.state
        x_pixel, y_pixel = y * self.tile_size, x * self.tile_size
        self.canvas.create_image(x_pixel, y_pixel, anchor=tk.NW, image=self.images["elf_down"])

    def epsilon_greedy(self, state):
        """Selecciona una acción usando ε-greedy."""
        if np.random.uniform(0, 1) < self.epsilon:
            return random.choice([0, 1, 2, 3])  # Exploración
        else:
            return np.argmax(self.q_table[state])  # Explotación

    def run_q_learning(self):
        """Ejecuta el algoritmo Q-Learning con decaimiento de ε."""
        for episode in range(self.episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False
            step = 0

            while not done:
                # Seleccionar acción usando ε-greedy
                action = self.epsilon_greedy(state_index)

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Actualizar la tabla Q usando la regla de Q-Learning
                self.q_table[state_index, action] += self.alpha * (
                    reward
                    + self.gamma * np.max(self.q_table[next_state_index])
                    - self.q_table[state_index, action]
                )

                # Actualizar estado
                state_index = next_state_index
                self.state = next_state
                done = terminated
                step += 1

                # Redibujar el mapa
                self.draw_map()
                self.root.update()

                # Verificar si se alcanzó el objetivo
                if terminated and reward > 0:
                    self.success_count += 1

            # Reducir ε después de cada episodio
            self.epsilon = max(self.epsilon_min, self.epsilon * self.decay)

            # Actualizar el historial de éxitos cada 100 episodios
            if (episode + 1) % 100 == 0:
                success_rate = self.success_count / 100
                self.success_history.append(success_rate)
                self.success_count = 0

        # Mostrar resultados finales
        self.message_label.config(text=f"Q-Learning completado. Tasa de éxito: {self.success_history[-1]:.2f}")

    def reset(self):
        """Reinicia el entorno."""
        self.state = self.lake.reset()
        self.q_table = np.zeros((self.lake.n_rows * self.lake.n_cols, 4))
        self.epsilon = 0.5  # Restaurar epsilon inicial
        self.current_episode = 0
        self.done = False
        self.success_count = 0
        self.success_history = []
        self.draw_map()
        self.message_label.config(text="Presiona 'Ejecutar Q-Learning con Decaimiento de ε' para comenzar.")

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Crear la ventana de Tkinter
root = tk.Tk()
root.title("FrozenLake - Ejercicio 5: Q-Learning con Decaimiento de ε")
app = DecayEpsilonQLearningVisualizer(root, lake, episodes=500, epsilon_0=0.5, epsilon_min=0.01, decay=0.995)
root.mainloop()


### **Ejercicio 6: Evaluación Apropiada**
- **Objetivo**:
  - Implementar un bucle interno para evaluar las políticas aprendidas repetidamente.
  - Comparar el rendimiento promedio entre SARSA y Q-Learning.
  - Concluir en qué circunstancias un algoritmo es mejor que el otro.



In [1]:
import numpy as np

class PolicyEvaluator:
    def __init__(self, lake, q_table_sarsa, q_table_qlearning, eval_episodes=100):
        self.lake = lake
        self.q_table_sarsa = q_table_sarsa
        self.q_table_qlearning = q_table_qlearning
        self.eval_episodes = eval_episodes

    def evaluate_policy(self, q_table):
        """Evalúa una política aprendida representada por una tabla Q."""
        success_count = 0
        total_reward = 0

        for _ in range(self.eval_episodes):
            state = self.lake.reset()
            state_index = state[0] * self.lake.n_cols + state[1]
            done = False

            while not done:
                # Seleccionar la mejor acción según la tabla Q
                action = np.argmax(q_table[state_index])

                # Tomar un paso en el entorno
                next_state, reward, terminated = self.lake.step(action)
                next_state_index = next_state[0] * self.lake.n_cols + next_state[1]

                # Acumular recompensa
                total_reward += reward

                # Verificar si el episodio fue exitoso
                if terminated and reward > 0:
                    success_count += 1

                # Actualizar estado
                state_index = next_state_index
                done = terminated

        # Calcular métricas
        success_rate = success_count / self.eval_episodes
        avg_reward = total_reward / self.eval_episodes
        return success_rate, avg_reward

    def compare_policies(self):
        """Compara las políticas aprendidas por SARSA y Q-Learning."""
        print("Evaluando política SARSA...")
        success_rate_sarsa, avg_reward_sarsa = self.evaluate_policy(self.q_table_sarsa)
        print(f"Tasa de éxito SARSA: {success_rate_sarsa:.2f}, Recompensa promedio: {avg_reward_sarsa:.2f}")

        print("Evaluando política Q-Learning...")
        success_rate_qlearning, avg_reward_qlearning = self.evaluate_policy(self.q_table_qlearning)
        print(f"Tasa de éxito Q-Learning: {success_rate_qlearning:.2f}, Recompensa promedio: {avg_reward_qlearning:.2f}")

        if success_rate_qlearning > success_rate_sarsa:
            print("\nConclusión: Q-Learning superó a SARSA en términos de tasa de éxito.")
        elif success_rate_qlearning < success_rate_sarsa:
            print("\nConclusión: SARSA superó a Q-Learning en términos de tasa de éxito.")
        else:
            print("\nConclusión: Ambas políticas tuvieron un rendimiento similar.")

# Ejemplo de uso

# Crear el entorno FrozenLake
lake = FrozenLake(is_slippery=True)

# Supongamos que tenemos tablas Q aprendidas para SARSA y Q-Learning
q_table_sarsa = np.random.rand(lake.n_rows * lake.n_cols, 4)  # Simulada para ejemplo
q_table_qlearning = np.random.rand(lake.n_rows * lake.n_cols, 4)  # Simulada para ejemplo

# Crear el evaluador
evaluator = PolicyEvaluator(lake, q_table_sarsa, q_table_qlearning, eval_episodes=100)

# Comparar las políticas
evaluator.compare_policies()


NameError: name 'FrozenLake' is not defined